In [1]:
#Import packages
import pandas as pd
import nltk
nltk.download('punkt')
nltk.download('vader_lexicon')
#from textblob import TextBlob
import pandas as pd
import numpy as np
from nltk.classify import NaiveBayesClassifier
from nltk.corpus import subjectivity
from nltk.sentiment import SentimentAnalyzer
from nltk.sentiment.util import *
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk import tokenize

[nltk_data] Downloading package punkt to /Users/kairuoyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/kairuoyan/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [2]:
#Question 3:

#Setting up columns for sentiment analysis
sentiment_analysis_columns = ['negativity', 'neutrality', 'positivity', 'compound']

file_names = ['reviews']

sid = SentimentIntensityAnalyzer()
#Sentiment analysis
for i in range(len(file_names)):
    expanded_text_dataset = []
    file_name = file_names[i]
    print('Sentiment analysis for: %s' %file_name)
    file_df = pd.read_csv('%s.csv' % file_name)
    column_names = list(file_df.columns.values)
    all_columns = column_names + sentiment_analysis_columns
    text_segment_list = list(file_df['comments'])
    for j in range(len(text_segment_list)):
        row_info = list(file_df.iloc[j])
        text = text_segment_list[j]
        ##Sentiment analysis
        try:
            ss = sid.polarity_scores(text)
            negativity = ss['neg']
            neutrality = ss['neu']
            positivity = ss['pos']
            compound = ss['compound']
            temp_data = row_info + [negativity, neutrality, positivity, compound]
            expanded_text_dataset.append(temp_data)
        except:
            temp_data = row_info + ['NA', 'NA', 'NA', 'NA']
            expanded_text_dataset.append(temp_data)
    ##Storing the results in a dataset
    sentiment_dataset_df = pd.DataFrame(expanded_text_dataset, columns = all_columns)
    sentiment_dataset_df.to_csv('%s.csv' %file_name, index = False)

Sentiment analysis for: reviews


In [3]:
#Question 4:
#Import new datasets
reviews_df = pd.read_csv("reviews.csv")
neg_df = pd.read_csv("negative_words.csv", encoding='cp1252', header = None)
pos_df = pd.read_csv("positive_words.csv", header= None)
reviews_df['comments'] = reviews_df['comments'].astype(str)

#Create two new columns for calculation
reviews_df['negativity_simple'] = None
reviews_df['positivity_simple'] = None

#Count total words, negative words, and positive words in each comment
for i in range(len(reviews_df['comments'])):
    cmt = reviews_df['comments'][i].split()
    pos_word = 0
    neg_word = 0
    for word in cmt:
        if word in pos_df.values:
            pos_word =+ 1
        if word in neg_df.values:
            neg_word =+ 1
    reviews_df['positivity_simple'][i] = pos_word/len(reviews_df['comments'][i])
    reviews_df['negativity_simple'][i] = neg_word/len(reviews_df['comments'][i])

/Users/kairuoyan/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/kairuoyan/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [4]:
#Question 5:
reviews_df = reviews_df.astype({'positivity_simple': 'float', 'negativity_simple': 'float'})
listings_df = pd.read_csv('listings.csv')
listings_df.set_index('listing_id', inplace= True)

#New columns for listings_df
col = ['negativity_mean', 'neutrality_mean', 'positivity_mean', 'compound_mean','positivity_simple_mean', 'negativity_simple_mean']

#Putting these new columns into listings_df
for i in col:
    score = reviews_df.groupby('listing_id').agg({i.replace('_mean', ''): 'mean'})
    score.rename(columns={i.replace('_mean', '') : i}, inplace=True)
    listings_df = listings_df.join(score, on='listing_id', how= 'outer')

In [5]:
#Create new CSV file for later use
listings_df.to_csv('listings_new.csv', index=False)